## Lab 5: Tensorflow Intro

#### CSC 215 Artificial Intelligence (Spring 2019)

#### Dr. Haiquan Chen, California State University, Sacramento

# Helpful Functions for Tensorflow (little gems)

The following functions will be used with TensorFlow to help preprocess the data.  They allow you to build the feature vector for a neural network. 

* Predictors/Inputs 
    * Fill any missing inputs with the median for that column.  Use **missing_median**.
    * Encode textual/categorical values with **encode_text_dummy**.
    * Encode numeric values with **encode_numeric_zscore**.
* Output
    * Discard rows with missing outputs.
    * Encode textual/categorical values with **encode_text_index**.
    * Do not encode output numeric values.
* Produce final feature vectors (x) and expected output (y) with **to_xy**.

In [1]:
import collections
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)



# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, collections.Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low


# Classification or Regression

Neural networks can function in *** classification or regression***:

* **Regression** - You expect a number as your neural network's prediction.
* **Classification** - You expect a class/category as your neural network's prediction.

The following shows a classification and regression neural network:

![Neural Network Classification and Regression](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/class_2_ann_class_reg.png "Neural Network Classification and Regression")


Regression networks always have a single output neuron.  Classification neural networks have an output neuron for each class. 

These neurons are grouped into layers:

* **Input Layer** - The input layer accepts feature vectors from the dataset.  Input layers usually have a bias neuron.
* **Output Layer** - The output from the neural network.  The output layer does not have a bias neuron.
* **Hidden Layers** - Layers that occur between the input and output layers.  Each hidden layer will usually have a bias neuron.


There are usually five types of neurons in a neural network:

* **Input Neurons** - *** Each input neuron is mapped to one element in the feature vector.***
* **Hidden Neurons** - Hidden neurons allow the neural network to abstract and process the input into the output.
* **Output Neurons** - Each output neuron calculates one part of the output.
* **Context Neurons** - Holds state in the neural network.
* **Bias Neurons** - Work similar to the y-intercept of a linear equation.  



# Activation Functions

### Historically neural networks have used a *** hyperbolic tangent***, ***sigmoid/logistic***, or ***linear function***.  

### However, modern deep neural networks primarily make use of the following activation functions:

* **Rectified Linear Unit (ReLU)** - Used for the output of hidden layers.
* **Softmax** - Used for the output of classification neural networks. 



(1) The ReLU function is calculated as follows:

$ \phi(x) = \max(0, x) $



(2) The Softmax is calculated as follows:

$ \phi_i(z) = \frac{e^{z_i}}{\sum\limits_{j \in group}e^{z_j}} $

The Softmax activation function is only useful with more than one output neuron.  It ensures that all of the output neurons sum to 1.0.  This makes it very useful for classification where it shows the probability of each of the classes as being the correct choice.

To experiment with the Softmax, click [here](http://www.heatonresearch.com/aifh/vol3/softmax.html).



# TensorFlow - Google

![TensorFlow](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/class_2_tensorflow.png "TensorFlow")

TensorFlow is an open source software library for machine learning in various kinds of perceptual and language understanding tasks. It is currently used for both research and production by different teams in many commercial Google products, such as speech recognition, Gmail, Google Photos, and search.

* [TensorFlow Homepage](https://www.tensorflow.org/)


# What version of TensorFlow do you have?

TensorFlow is very new and changing rapidly. 

In [3]:
import tensorflow as tf
print("Tensor Flow Version: {}".format(tf.__version__))

Tensor Flow Version: 1.10.0


# Why TensorFlow

* Supported by Google
* Works well on Linux/Mac
* Excellent GPU support
* Most popular today

# Using Keras

[Keras](https://keras.io/) is an official high-end API, supported by Google, for Tensorflow that makes it much easier to create neural networks.    

Keras is a separate install from TensorFlow.  

To install Keras, use **pip install keras** after **pip install tensorflow**.

# Example of TensorFlow Regression: MPG Prediction

This example shows how to encode the MPG dataset for regression.  Notice that:

* Input has both numeric and categorical
* Input has missing values

In [4]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
path = "./data/"

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

df[0:5]

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [5]:
cars = df['name']

df.drop('name',1,inplace=True)

missing_median(df, 'horsepower')

encode_text_dummy(df, 'origin')

x,y = to_xy(df,"mpg")

In [10]:
x.shape

(398, 9)

In [11]:
y.shape

(398,)

In [12]:
x

array([[  8., 307., 130., ...,   1.,   0.,   0.],
       [  8., 350., 165., ...,   1.,   0.,   0.],
       [  8., 318., 150., ...,   1.,   0.,   0.],
       ...,
       [  4., 135.,  84., ...,   1.,   0.,   0.],
       [  4., 120.,  79., ...,   1.,   0.,   0.],
       [  4., 119.,  82., ...,   1.,   0.,   0.]], dtype=float32)

In [13]:
y

array([18. , 15. , 18. , 16. , 17. , 15. , 14. , 14. , 14. , 15. , 15. ,
       14. , 15. , 14. , 24. , 22. , 18. , 21. , 27. , 26. , 25. , 24. ,
       25. , 26. , 21. , 10. , 10. , 11. ,  9. , 27. , 28. , 25. , 25. ,
       19. , 16. , 17. , 19. , 18. , 14. , 14. , 14. , 14. , 12. , 13. ,
       13. , 18. , 22. , 19. , 18. , 23. , 28. , 30. , 30. , 31. , 35. ,
       27. , 26. , 24. , 25. , 23. , 20. , 21. , 13. , 14. , 15. , 14. ,
       17. , 11. , 13. , 12. , 13. , 19. , 15. , 13. , 13. , 14. , 18. ,
       22. , 21. , 26. , 22. , 28. , 23. , 28. , 27. , 13. , 14. , 13. ,
       14. , 15. , 12. , 13. , 13. , 14. , 13. , 12. , 13. , 18. , 16. ,
       18. , 18. , 23. , 26. , 11. , 12. , 13. , 12. , 18. , 20. , 21. ,
       22. , 18. , 19. , 21. , 26. , 15. , 16. , 29. , 24. , 20. , 19. ,
       15. , 24. , 20. , 11. , 20. , 21. , 19. , 15. , 31. , 26. , 32. ,
       25. , 16. , 16. , 18. , 16. , 13. , 14. , 14. , 14. , 29. , 26. ,
       26. , 31. , 32. , 28. , 24. , 26. , 24. , 26

In [14]:
model = Sequential()

model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1     #  why input_dim=x.shape[1]?  
model.add(Dense(10, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output

model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(x,y,verbose=2,epochs=100)    # Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.


Epoch 1/100
 - 0s - loss: 1429886.5336
Epoch 2/100
 - 0s - loss: 542954.2550
Epoch 3/100
 - 0s - loss: 130569.2813
Epoch 4/100
 - 0s - loss: 12620.5755
Epoch 5/100
 - 0s - loss: 1111.2165
Epoch 6/100
 - 0s - loss: 2217.2666
Epoch 7/100
 - 0s - loss: 862.3796
Epoch 8/100
 - 0s - loss: 355.2959
Epoch 9/100
 - 0s - loss: 342.6379
Epoch 10/100
 - 0s - loss: 325.7177
Epoch 11/100
 - 0s - loss: 314.7933
Epoch 12/100
 - 0s - loss: 312.1185
Epoch 13/100
 - 0s - loss: 310.2586
Epoch 14/100
 - 0s - loss: 308.1708
Epoch 15/100
 - 0s - loss: 306.3058
Epoch 16/100
 - 0s - loss: 304.4288
Epoch 17/100
 - 0s - loss: 302.2201
Epoch 18/100
 - 0s - loss: 299.9109
Epoch 19/100
 - 0s - loss: 297.8357
Epoch 20/100
 - 0s - loss: 295.3646
Epoch 21/100
 - 0s - loss: 293.1278
Epoch 22/100
 - 0s - loss: 290.9215
Epoch 23/100
 - 0s - loss: 288.7957
Epoch 24/100
 - 0s - loss: 286.3107
Epoch 25/100
 - 0s - loss: 283.8220
Epoch 26/100
 - 0s - loss: 281.4275
Epoch 27/100
 - 0s - loss: 278.5256
Epoch 28/100
 - 0s - lo

### Monitor the loss at each epoch

One line is produced for each training epoch.  You can eliminate this output by setting the verbose setting of the fit command:

* **verbose=0** - No progress output (use with Juputer if you do not want output)
* **verbose=1** - Display progress bar, does not work well with Jupyter
* **verbose=2** - Summary progress output (use with Jupyter if you want to know the loss at each epoch)


## Use Trained Model to Make Regression Prediction

Next we will perform actual predictions.  These predictions are assigned to the **pred** variable. These are all MPG predictions from the neural network.  

***Notice that the data to predict should be a 2D array!***  

***Notice that the prediction result is also a 2D array!*** 

Neural networks can return multiple values, so the result is always an array.  Here the neural network only returns 1 value per prediction (there are 398 cars, so 398 predictions).  However, a 2D array is needed because the neural network has the potential of returning more than one value. 


In [15]:
pred = model.predict(x)
print("Shape: {}".format(pred.shape))
print(pred)

Shape: (398, 1)
[[20.100216]
 [24.749874]
 [23.356075]
 [24.14288 ]
 [21.942501]
 [26.776974]
 [29.758358]
 [29.294064]
 [30.814877]
 [27.071896]
 [24.224056]
 [23.495846]
 [19.350765]
 [32.52093 ]
 [22.41647 ]
 [18.71716 ]
 [19.136227]
 [17.148983]
 [21.90295 ]
 [15.807949]
 [21.399899]
 [21.717007]
 [23.54919 ]
 [25.363674]
 [17.84097 ]
 [33.333187]
 [33.090878]
 [34.221493]
 [32.21356 ]
 [21.994656]
 [20.226254]
 [22.463406]
 [23.637997]
 [18.103024]
 [18.88409 ]
 [17.075191]
 [14.809811]
 [17.840694]
 [24.298641]
 [23.795345]
 [22.3717  ]
 [23.063717]
 [24.801449]
 [22.591854]
 [23.04803 ]
 [18.685848]
 [17.392056]
 [17.021053]
 [14.788998]
 [19.886257]
 [21.729122]
 [20.679989]
 [20.343136]
 [20.098873]
 [20.801449]
 [18.241909]
 [19.869625]
 [22.834103]
 [20.60345 ]
 [17.676113]
 [21.048   ]
 [20.541317]
 [24.307491]
 [24.101864]
 [23.219723]
 [22.380978]
 [23.907955]
 [28.73376 ]
 [22.442867]
 [23.459835]
 [27.043758]
 [24.074337]
 [23.855953]
 [19.983578]
 [22.305294]
 [23.4079

We would like to see how good these predictions are.  We know what the correct MPG is for each car, so we can measure how close the neural network was.

In [16]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 9.825636863708496


We can also print out the first 10 cars, with predictions and actual MPG.

In [17]:
# Sample predictions
for i in range(10):
    print("{}. Car name: {}, MPG: {}, predicted MPG: {}".format(i+1,cars[i],y[i],pred[i]))

1. Car name: chevrolet chevelle malibu, MPG: 18.0, predicted MPG: [20.100216]
2. Car name: buick skylark 320, MPG: 15.0, predicted MPG: [24.749874]
3. Car name: plymouth satellite, MPG: 18.0, predicted MPG: [23.356075]
4. Car name: amc rebel sst, MPG: 16.0, predicted MPG: [24.14288]
5. Car name: ford torino, MPG: 17.0, predicted MPG: [21.942501]
6. Car name: ford galaxie 500, MPG: 15.0, predicted MPG: [26.776974]
7. Car name: chevrolet impala, MPG: 14.0, predicted MPG: [29.758358]
8. Car name: plymouth fury iii, MPG: 14.0, predicted MPG: [29.294064]
9. Car name: pontiac catalina, MPG: 14.0, predicted MPG: [30.814877]
10. Car name: amc ambassador dpl, MPG: 15.0, predicted MPG: [27.071896]


# Example of TensorFlow Classification: Iris

This is a very simple example of how to perform the Iris classification using TensorFlow. The iris.csv file is used. 


In [18]:
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping


df=pd.read_csv("data/iris.csv",na_values=['NA','?'])

species = encode_text_index(df,"species")

x,y = to_xy(df,"species")

In [19]:
x

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [20]:
x.shape

(150, 4)

In [21]:
y  #  This is one-hot encoding.  Only one value is 1.0 (hot)

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

In [22]:
y.shape

(150, 3)

In [23]:
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(y.shape[1], activation='softmax')) # Output

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(x,y,verbose=2,epochs=100)

Epoch 1/100
 - 0s - loss: 1.1947
Epoch 2/100
 - 0s - loss: 1.0867
Epoch 3/100
 - 0s - loss: 1.0027
Epoch 4/100
 - 0s - loss: 0.9301
Epoch 5/100
 - 0s - loss: 0.8637
Epoch 6/100
 - 0s - loss: 0.8103
Epoch 7/100
 - 0s - loss: 0.7648
Epoch 8/100
 - 0s - loss: 0.7226
Epoch 9/100
 - 0s - loss: 0.6806
Epoch 10/100
 - 0s - loss: 0.6445
Epoch 11/100
 - 0s - loss: 0.6133
Epoch 12/100
 - 0s - loss: 0.5840
Epoch 13/100
 - 0s - loss: 0.5570
Epoch 14/100
 - 0s - loss: 0.5268
Epoch 15/100
 - 0s - loss: 0.5028
Epoch 16/100
 - 0s - loss: 0.4742
Epoch 17/100
 - 0s - loss: 0.4480
Epoch 18/100
 - 0s - loss: 0.4233
Epoch 19/100
 - 0s - loss: 0.3984
Epoch 20/100
 - 0s - loss: 0.3793
Epoch 21/100
 - 0s - loss: 0.3555
Epoch 22/100
 - 0s - loss: 0.3387
Epoch 23/100
 - 0s - loss: 0.3234
Epoch 24/100
 - 0s - loss: 0.3047
Epoch 25/100
 - 0s - loss: 0.2903
Epoch 26/100
 - 0s - loss: 0.2750
Epoch 27/100
 - 0s - loss: 0.2626
Epoch 28/100
 - 0s - loss: 0.2502
Epoch 29/100
 - 0s - loss: 0.2383
Epoch 30/100
 - 0s - lo

In [24]:
# Print out number of species found:
print(species)

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


Now that you have a neural network trained, we would like to be able to use it. There were 3 types of iris (Iris-setosa, Iris-versicolor, and Iris-virginica).  

In [25]:
pred = model.predict(x)
pred.shape

(150, 3)

In [26]:
pred

array([[9.99054611e-01, 9.45410051e-04, 4.59877825e-09],
       [9.97352600e-01, 2.64743809e-03, 3.38287940e-08],
       [9.98359025e-01, 1.64100237e-03, 1.95799021e-08],
       [9.96380270e-01, 3.61958100e-03, 6.88471289e-08],
       [9.99151945e-01, 8.48067051e-04, 4.16093116e-09],
       [9.98527288e-01, 1.47267454e-03, 4.74248996e-09],
       [9.97869730e-01, 2.13030446e-03, 2.82263191e-08],
       [9.98426795e-01, 1.57318555e-03, 1.10275273e-08],
       [9.95509267e-01, 4.49057482e-03, 1.35407490e-07],
       [9.97744083e-01, 2.25594384e-03, 2.44399594e-08],
       [9.99302268e-01, 6.97715499e-04, 1.83004911e-09],
       [9.97528970e-01, 2.47104419e-03, 2.61355151e-08],
       [9.97774661e-01, 2.22532335e-03, 2.92666531e-08],
       [9.98561442e-01, 1.43859966e-03, 3.00720622e-08],
       [9.99858975e-01, 1.41047756e-04, 1.07036491e-10],
       [9.99727666e-01, 2.72372155e-04, 2.30900327e-10],
       [9.99528885e-01, 4.71155101e-04, 9.99804572e-10],
       [9.98793840e-01, 1.20608

In [27]:
# print y.  
print(y)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0.

#### The column (pred) with the highest probability is  the prediction of the neural network.  

#### Use argmax function to find the index of the maximum prediction for each row.

In [28]:
# Usually the column (pred) with the highest prediction is considered to be the prediction of the neural network.  It is easy
# to convert the predictions to the expected iris species.  The argmax function finds the index of the maximum prediction
# for each row.

predict_classes = np.argmax(pred,axis=1)

true_classes = np.argmax(y,axis=1)

print("Predictions: {}".format(predict_classes))
print("True: {}".format(true_classes))

Predictions: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
True: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [29]:
# Of course it is very easy to turn these indexes back into iris species.  We just use the species list that we created earlier.
print(species[predict_classes[0:10]])

['Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa']


In [30]:

#For all of the iris predictions, what percent were correct?  

correct = metrics.accuracy_score(true_classes, predict_classes)
print("Accuracy: {}".format(correct))

Accuracy: 0.98


The code below performs two ad hoc predictions.  

*** Remember x should be a 2D array! ***

In [31]:
# ad hoc prediction
sample_flower = np.array( [[5.0,3.0,4.0,2.0]], dtype=float)
pred = model.predict(sample_flower)
print(pred)

[[3.7992137e-04 2.3699242e-01 7.6262760e-01]]


In [32]:
pred = np.argmax(pred, axis=1)
print("Predict that {} is: {}".format(sample_flower,species[pred]))

Predict that [[5. 3. 4. 2.]] is: ['Iris-virginica']


Notice that the argmax in the second prediction requires **axis=1**?  Since we have a 2D array now, we must specify which axis to take the argmax over.  The value **axis=1** specifies we want the max column index for each row.

In [33]:
# predict two sample flowers
sample_flower = np.array( [[5.0,3.0,4.0,2.0],[5.2,3.5,1.5,0.8]], dtype=float)
pred = model.predict(sample_flower)
print(pred)

[[3.7992137e-04 2.3699242e-01 7.6262760e-01]
 [9.9276841e-01 7.2315307e-03 9.3080367e-08]]


In [34]:
pred = np.argmax(pred, axis=1)
print("Predict that {} is: {}".format(sample_flower,species[pred]))

Predict that [[5.  3.  4.  2. ]
 [5.2 3.5 1.5 0.8]] is: ['Iris-virginica' 'Iris-setosa']


# Load/Save Trained Network

Complex neural networks will take a long time to fit/train.  It is helpful to be able to save these neural networks so that they can be reloaded later.  A reloaded neural network will not require retraining.  Keras provides three formats for neural network saving.

* **YAML** - Stores the neural network structure (no weights) in the [YAML file format](https://en.wikipedia.org/wiki/YAML).
* **JSON** - Stores the neural network structure (no weights) in the [JSON file format](https://en.wikipedia.org/wiki/JSON).
* **HDF5** - Stores the complete neural network (with weights) in the [HDF5 file format](https://en.wikipedia.org/wiki/Hierarchical_Data_Format). 

Usually you will want to save in HDF5.

In [32]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
path = "./data/"
save_path = "./dnn/"

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

cars = df['name']
df.drop('name',1,inplace=True)
missing_median(df, 'horsepower')
x,y = to_xy(df,"mpg")
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x,y,verbose=0,epochs=100)

# Predict
pred = model.predict(x)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print("Before save score (RMSE): {}".format(score))


# save entire network to HDF5 (save everything)
model.save(os.path.join(save_path,"network.h5"))

Before save score (RMSE): 5.817868232727051


Now we reload the network and perform another prediction.  The RMSE should match the previous one exactly if the neural network was really saved and reloaded.

In [33]:
from keras.models import load_model
model2 = load_model(os.path.join(save_path,"network.h5"))
pred = model2.predict(x)
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print("After load score (RMSE): {}".format(score))

After load score (RMSE): 5.817868232727051


### References:

* [Google Colab](https://colab.research.google.com/) - Free web based platform that includes Python, Juypter Notebooks, and TensorFlow with free GPU support.  No setup needed.
* [IBM Cognitive Class Labs](https://www.datascientistworkbench.com) - Free web based platform that includes Python, Juypter Notebooks, and TensorFlow.  No setup needed.
* [Python Anaconda](https://www.continuum.io/downloads) - Python distribution that includes many data science packages, such as Numpy, Scipy, Scikit-Learn, Pandas, and much more.
* [TensorFlow](https://www.tensorflow.org/) - Google's mathematics package for deep learning.
* [Kaggle](https://www.kaggle.com/) - Competitive data science.  Good source of sample data.
* T81-558: Applications of Deep Neural Networks. Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/)